This is the template for DS3000 Final data analysis project. Once you finish, please remove all my instructions. You do not need to exactly follow the structure in the template but please make sure you have all the components. Write your report in paragraphs. Only use bullet points when list something (eg: functions) 

# Investigating Palworld Through Data Analysis
#### Team 11
- Ansh Aggarwal
- Sohum Balsara
- Bear Smith

In [22]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import plotly.express as px

## Introduction

In this project we study Palworld, a survival and monster-taming video game in which players collect characters called Pals to battle each other and help build their bases. Each Pal has a different set of skills and qualifications, a combination of such features as their element type, their HP in battle, and their "work suitability" for various tasks like farming or electricity production. We want to learn about how those factors combine and influence each other, and what relationships that may not be stated outwardly can be found between them. This may help players better understand their game and be more successful at it, or at least increase their knowledge of the logic behind Palworld.

Our primary research questions are:
- How are a Pal's work suitability scores correlated to its other statistics, such as its element type or battle skills?
- What relationship does Pal rarity have with their other statistics?

## Data 

### Data Source

- All data was taken from [palworld.gg]("https://palworld.gg").
- We scraped each Pal's name, ID number, rarity, element(s), work suitability type(s) and level(s), HP, and defense scores.
- To make the data frame more easily filtered, the Pals' elements were listed using multiple boolean columns, each representing one element.
- For that same reason, their work suitabilities were also listed by column, but instead of True or False, each value was the Pal's level in that suitability or Nan if they did not have it. That way, dropna can be used to filter the data frame by element type.
- Finally, the Pals' HP and defense stats were converted to numbers in the data frame, so that mathematical operations can be done on those columns.

### Webscraping and cleaning functions overview
- `scrape_pal_ids()`
    - Creates a data frame with all the Pals' names as its index and one column of data containing their IDs.
- `scrape_pal_rarity()`
    - Adds a column containing the rarity of each Pal to the given data frame.
- `scrape_pal_elements()`
    - Adds the element or elements of each Pal to the given data frame using one-hot columns.
- `scrape_pal_work()`
    - Adds the work suitability or suitabilities of each Pal to the given data frame, along with its level.
- `scrape_pal_hp()`
    - Adds a column containing the HP of each Pal to the given data frame.
- `scrape_pal_defense()`
    - Adds a column containing the defense score of each Pal to the given data frame.

### Data overview

# TODO - IN PROGRESS
- You can see a couple rows of our data below.
- We store the Pals' rarity, their element or elements, their work suitabilities and what levels they are, 
- Discuss if there is any potential problems about the data (eg: missing values, any features that you did not collect but may be important, any other concerns)

In [ ]:
import pandas as pd
pal_frame = pd.read_excel("pal_frame_for_project.xlsx")
# Reset the index back to the Pal names after reading in the spreadsheet file.
pal_frame.set_index("Unnamed: 0", inplace = True)
pal_frame.rename_axis(None, axis = 0, inplace = True)
pal_frame.head()

,ID,Rarity,Earth,Fire,Dragon,Dark,Electricity,Water,Ice,Leaf,...,Kindling,Gathering,Generating Electricity,Watering,Cooling,Farming,Medicine Production,Planting,HP,Defense
Anubis,#100,Epic,True,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120,100
Arsox,#42,Common,False,True,False,False,False,False,False,False,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,95
Astegon,#98,Epic,False,False,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,125
Azurmane,#136,Rare,False,False,False,False,True,False,False,False,...,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,130,110
Azurobe,#82,Rare,False,False,True,False,False,True,False,False,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,110,100


## Webscraping and cleaning

In [24]:
# list all the functions you have for webscraping and cleaning. Make sure write full 
# docstrings for each function

def scrape_pal_ids(url = "https://palworld.gg/pals"):
    """
    Scrape Pal IDs from the given URL.

    Args:
        url (str): The URL of the Palworld database page, assumed to be the current link unless otherwise specified.

    Returns:
        DataFrame: A dataFrame of Pal names and IDs.
    """
    response = requests.get(url).text
    soup = BeautifulSoup(response)

    # Find all Pal ID elements
    pal_dict = {}
    for pal in soup.find_all("div", class_ = "pal"):

        # Remove empty Pal entries.
        if pal.attrs["style"] == "display:none;":
            continue

        # Get the ID and name of the Pal and add them to the dictionary.
        pal_id_element = pal.find('span', class_='index').text.strip()
        pal_dict[pal.find("div", class_ = "name").next_element.strip()] = pal_id_element

    # Convert the dictionary to a DataFrame before returning it.
    return pd.DataFrame().from_dict(pal_dict, orient = "index", columns = ["ID"])

In [25]:
def scrape_pal_rarity(pal_df, url = "https://palworld.gg/pals"):
    """
    Scrape the rarity of each Pal listed in the database.

    Args:
        url (str): The URL of the database page, assumed to be the current link unless otherwise specified.
        pal_df (DataFrame): The DataFrame of Pal data to update.

    Returns:
        pal_df (DataFrame): The DataFrame given, with the rarity of each Pal added under a new column.
    """
    soup = BeautifulSoup(requests.get(url).text)

    # Find all Pals, then find each of their rarities.
    pal_tag = soup.find_all("div", class_ = "pal")
    for pal in pal_tag:

        # Remove empty Pal entries.
        if pal.attrs["style"] == "display:none;":
            continue

        # The "name" class is used twice in each Pal entry, first for their name, then for their rarity.
        name_class = pal.find_all("div", class_ = "name")
        # (Using next_element here rather than .text to avoid also getting the text from the nested children.)
        # Add the rarity of the Pal to the DataFrame under its name.
        pal_df.loc[name_class[0].next_element.strip(), "Rarity"] = name_class[1].next_element

    return pal_df


In [26]:
def scrape_pal_elements(pal_df, url = "https://palworld.gg/pals"):
    """
    Scrape the element or elements of each Pal listed in the database.

    Args:
        url (str): The URL of the database page, assumed to be the current link unless otherwise specified.
        pal_df (DataFrame): The DataFrame of Pal data to update.

    Returns:
        pal_df (DataFrame): The DataFrame given, with a dummy variable added for each element,
        True meaning that a Pal does belong to them and False for not.
    """
    soup = BeautifulSoup(requests.get(url).text)

    # Find each Pal entry.
    pal_tag = soup.find_all("div", class_ = "pal")
    for pal in pal_tag:
        # Remove empty Pal entries.
        if pal.attrs["style"] == "display:none;":
            continue

        # Load the individual entry page for the Pal.
        pal_page = requests.get("https://palworld.gg" + pal.a.attrs["href"]).text
        pal_soup = BeautifulSoup(pal_page)

        # Scrape the Pal's name.
        pal_name = pal_soup.find("h1", class_ = "name").text.strip()

        # Initialize all the element column values for this Pal to False, as none have been found yet.
        pal_df.loc[pal_name, ["Earth", "Fire", "Dragon", "Dark", "Electricity", "Water", "Ice", "Leaf", "Normal"]] = False

        # Scrape the elements of the Pal.
        pal_elems_tags = pal_soup.find("div", class_ = "elements").find_all("div", class_ = "name")
        for tag in pal_elems_tags:
            # For each found element, change the Pal's value in the corresponding column to True.
            pal_df.loc[pal_name, tag.text] = True
        
    return pal_df


In [27]:
def scrape_pal_work(pal_df, url = "https://palworld.gg/pals"):
    """
    Scrape the work suitabilities of each Pal listed in the database, including the level of each work type.

    Args:
        url (str): The URL of the database page, assumed to be the current link unless otherwise specified.
        pal_df (DataFrame): The DataFrame of Pal data to update.

    Returns:
        pal_df (DataFrame): The DataFrame given, with 13 new columns added: 12 representing the work suitabilities
        of Pals, each filled with NaN or the Pal's skill level for that task, and one column counting the number of suitabilities
        the Pal has in total.
    """
    soup = BeautifulSoup(requests.get(url).text)

    # Find each Pal entry.
    pal_tag = soup.find_all("div", class_ = "pal")
    for pal in pal_tag:
        # Remove empty Pal entries.
        if pal.attrs["style"] == "display:none;":
            continue

        # Load the individual entry page for the Pal.
        pal_page = requests.get("https://palworld.gg" + pal.a.attrs["href"]).text
        pal_soup = BeautifulSoup(pal_page)

        # Scrape the name of the Pal.
        pal_name = pal_soup.find("h1", class_ = "name").text.strip()

        # Scrape the suitabilities of the Pal, and for each one that is found add its level to the column with the suitability's name.
        pal_work_tags = pal_soup.find("div", class_ = "works").find_all("div", class_ = "active item")
        for tag in pal_work_tags:
            pal_df.loc[pal_name, tag.find("div", class_ = "name").text] = tag.find("span", class_ = "value").text
        # Add the pal's total number of suitabilities as a new column.
        pal_df.loc[pal_name, "Number of work suitabilities"] = len(pal_work_tags)
        
    return pal_df


In [28]:
def scrape_pal_hp(pal_df, url = "https://palworld.gg/pals"):
    """
    Scrape the HP of each Pal listed in the database.

    Args:
        url (str): The URL of the database page, assumed to be the current link unless otherwise specified.
        pal_df (DataFrame): The DataFrame of Pal data to update.

    Returns:
        pal_df (DataFrame): The DataFrame given, with a new column representing the Pals' HPs.
    """
    soup = BeautifulSoup(requests.get(url).text)

    # Find each Pal entry.
    pal_tag = soup.find_all("div", class_ = "pal")
    for pal in pal_tag:
        # Remove empty Pal entries.
        if pal.attrs["style"] == "display:none;":
            continue

        # Load the individual entry page for the Pal.
        pal_page = requests.get("https://palworld.gg" + pal.a.attrs["href"]).text
        pal_soup = BeautifulSoup(pal_page)

        # Scrape the name of the Pal.
        pal_name = pal_soup.find("h1", class_ = "name").text.strip()

        # Scrape the HP of the pal (the first entry in the stats section) and add it to the HP column.
        pal_hp = int(pal_soup.find("div", class_ = "stats").find_all("div", class_ = "value")[0].text)
        pal_df.loc[pal_name, "HP"] = pal_hp
        
    return pal_df


In [29]:
def scrape_pal_defense(pal_df, url = "https://palworld.gg/pals"):
    """
    Scrape the defense score of each Pal listed in the database.

    Args:
        url (str): The URL of the database page, assumed to be the current link unless otherwise specified.
        pal_df (DataFrame): The DataFrame of Pal data to update.

    Returns:
        pal_df (DataFrame): The DataFrame given, with a new column representing the Pals' defense scores.
    """
    soup = BeautifulSoup(requests.get(url).text)

    # Find each Pal entry.
    pal_tag = soup.find_all("div", class_ = "pal")
    for pal in pal_tag:
        # Remove empty Pal entries.
        if pal.attrs["style"] == "display:none;":
            continue

        # Load the individual entry page for the Pal.
        pal_page = requests.get("https://palworld.gg" + pal.a.attrs["href"]).text
        pal_soup = BeautifulSoup(pal_page)

        # Scrape the name of the Pal.
        pal_name = pal_soup.find("h1", class_ = "name").text.strip()

        # Scrape the defense of the pal (the second entry in the stats section) and add it to the defense column.
        pal_hp = int(pal_soup.find("div", class_ = "stats").find_all("div", class_ = "value")[1].text)
        pal_df.loc[pal_name, "Defense"] = pal_hp
        
    return pal_df


In [30]:
pal_df = scrape_pal_defense(scrape_pal_hp(scrape_pal_work(scrape_pal_elements(scrape_pal_rarity(scrape_pal_ids())))))
pal_df

,ID,Rarity,Earth,Fire,Dragon,Dark,Electricity,Water,Ice,Leaf,...,Kindling,Gathering,Generating Electricity,Watering,Cooling,Farming,Medicine Production,Planting,HP,Defense
Anubis,#100,Epic,True,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,100.0
Arsox,#42,Common,False,True,False,False,False,False,False,False,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,95.0
Astegon,#98,Epic,False,False,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,125.0
Azurmane,#136,Rare,False,False,False,False,True,False,False,False,...,NaN,1,4,NaN,NaN,NaN,NaN,NaN,130.0,110.0
Azurobe,#82,Rare,False,False,True,False,False,True,False,False,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,110.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wumpo Botan,#91,Epic,False,False,False,False,False,False,False,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,140.0,110.0
Xenogard,#126,Epic,False,False,True,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.0,130.0
Xenolord,#127,Epic,False,False,True,True,False,False,False,False,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,130.0,120.0
Xenovader,#125,Rare,False,False,False,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,85.0


## Visualizations

### Visualization functions overview
List all the functions you have written for visualization. For each one, write one sentence to describe it. 
- `make_hist()`
    - Generate a histogram with given data and feature
 
### Visualization results
- Present 3-4 data visualizations.
- For each visualization, you need to include title, xlabel, ylabel, legend (if necessary)
- For each visualization, explain why you make this data visualization (how it related to your research question) and explain what you have learned from this visualization

In [31]:
# list all the functions you have for visualization. Make sure write full 
# docstrings for each function
def make_hist(df, y_feat):

    pass

#### visualization 1

In [32]:
# Write the code to run functions to get each data visualization in separate code chunks. 
# Interpret the figures. 

#### visualization 2

In [33]:
# Write the code to run functions to get each data visualization in separate code chunks. 
# Interpret the figures. 

#### visualization 3

In [34]:
# Write the code to run functions to get each data visualization in separate code chunks. 
# Interpret the figures. 

## Models

### Modeling functions overview
List all the functions you have written for modeling. For each one, write one sentence to describe it. 
- `fit_linear()`
    - fit a linear model to the data and output the r2, slope and intercept

### Model results

- Present 2-3 models for the analysis.
- Explain any pre-processing steps you have done (eg: scaling, polynomial, dummy features)
- For each model, explain why you think this model is suitable and what metrics you want to use to evaluate the model
    - If it is a classification model, you need to present the confusion matrix, calculate the accuracy, sensitivity and specificity with cross-validation
    - If it is a regression model, you need to present the r2 and MSE with cross-validation
    - If it is a linear regression model/multiple linear regression model, you need to interpret the meaning of the coefficient with the full data
    - If it is a decision tree model, you need to plot the tree with the full data
    - If it is a random forest model, you need to present the feature importance plot with the full data
    - If it is a PCA, you need to explain how to select the number of components and interpret the key features in the first two components
    - If it is a clustering, you need explain how to select the number of clustering and summarize the clustering. 

In [35]:
# list all the functions you have for modeling. Make sure write full 
# docstrings for each function
def fit_linear(df, y_feat, x_feat):

    pass

#### Model 1

In [36]:
# Write the code to run functions to fit each model in separate code chunks. 
# Interpret the model results. 

#### Model 2

In [37]:
# Write the code to run functions to fit each model in separate code chunks. 
# Interpret the model results. 

#### Model 3

In [38]:
# Write the code to run functions to fit each model in separate code chunks. 
# Interpret the model results. 

## Discussion

- One or two paragraphs to summarize your findings in the modeling sections and do the models answer your research question?
- Any other potential thing you can do with the analysis (eg: include more features, get more data, try some other models etc.)
- List the contribution for each group member.

### Contributions
- Sohum came up with the majority of our research questions and visualization ideas.
- Bear wrote the web scraping functions.
- Ansh wrote the data visualizations.
- 